# EEVE-10.8B QLoRA Fine Tuning

## 1. Install Modules

In [1]:
!pip install -U datasets
!pip install -U bitsandbytes
!pip install -U accelerate
!pip install -U peft
!pip install -U trl
!pip install -U typing_extensions
!pip install -U torch
!pip install -U datasets
!pip install -U korouge_score
!pip install -U konlpy
!pip install -U wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 1.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 14.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of huggingface-hub to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 73.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## 2. Import Modules

In [2]:
import os
import torch
import transformers
from datasets import load_dataset
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline,
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel,
    PeftConfig,
)
from trl import SFTTrainer
from korouge_score import rouge_scorer
import numpy as np

os.environ["WANDB_PROJECT"] = "FinPilot"  # W&B 프로젝트 이름 지정
os.environ["WANDB_LOG_MODEL"] = "checkpoint"  # 모든 모델 체크포인트 로깅

/usr/local/lib/python3.10/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.10/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [ ]:
from huggingface_hub import login
login(token="")

In [4]:
wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: martinus-choi (FinPilot) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


## 3. Load Dataset

In [5]:
from datasets import load_dataset

train_dataset = load_dataset("MartinusChoi/FinPilot", split="train")
test_dataset = load_dataset("MartinusChoi/FinPilot", split="test")

train_dataset.csv:   0%|          | 0.00/908k [00:00<?, ?B/s]

test_dataset.csv:   0%|          | 0.00/93.4k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/406 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/42 [00:00<?, ? examples/s]

## 4. Set PLM Into QLoRA Form

In [ ]:
model_name = "yanolja/EEVE-Korean-Instruct-10.8B-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                     # Load model in 4bit precision
    bnb_4bit_quant_type='nf4',             # Pre-trained model has to be quantization in 4bit nf type
    bnb_4bit_use_double_quant=True,        # Use double-qauntization of QLoRA
    bnb_4bit_compute_dtype=torch.bfloat16  # Pre-trained model has to be loaded in BF16 dtype
)

plm = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=True,
    quantization_config=bnb_config,        # Use bitsandbytes config
    device_map='auto',                     # auto : HF Accelerate determines which GPU to allocate for each layer of the model.
    trust_remote_code=True                 # Setting for use EEVE model
)

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('yanolja/EEVE-Korean-Instruct-10.8B-v1.0', token=True,)
tokenizer.pad_token = tokenizer.eos_token

## 5. Set Hyper Parameters

In [ ]:
flm = prepare_model_for_kbit_training(plm)

lora_alpha = 32
lora_dropout = 0.05
lora_rank = 32

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias='none',
    task_type='CAUSAL_LM',
    target_modules=['q_proj','k_proj','v_proj','o_proj','gate_proj']
)

flm=get_peft_model(flm, peft_config)
flm.print_trainable_parameters()

In [ ]:
training_arguments =  transformers.TrainingArguments(
    output_dir = './train_output',
    per_device_train_batch_size = 2,
    gradient_accumulation_steps = 2,                                               # 배치 크기가 줄어들면 기울기 누적 단계가 2배 증가
    optim = 'paged_adamw_32bit',                                                   # 더 나은 메모리 관리를 위해 페이징을 활성화
    save_strategy='steps',                                                         # 학습 중에 채택할 체크포인트 save strategy
    save_steps = 10,                                                               # 두 개의 체크포인트가 저장되기 전의 업데이트 단계 수
    logging_steps = 5,                                                            # 두 로그 사이의 업데이트 단계 수
    learning_rate = 2e-4,                                                          # AdamW 최적화 프로그램의 학습률
    max_grad_norm = 0.3,                                                           # 최대 그라데이션 표준(gradient clipping)
    max_steps = 500,                                                                # 60 단계 동안 학습
    warmup_ratio = 0.03,                                                           # 0 에서 learning_rate 까지 선형 준비에 사용되는 단계 수
    lr_scheduler_type = 'cosine',                                                  # 학습률 스케줄러
    report_to = 'wandb',                                                           # You can find your API key in your browser here: https://wandb.ai/authorize
)

## 6. Fine Tuning

In [ ]:
def compute_metrics(eval_pred):

  # Dict for store 'rouge' score per predicted summary
  scores = {
      "rouge1": [],
      "rouge2": [],
      "rougeL": [],
      "rougeLsum": []
  }

  # 한국어 rouge score 계산 인스턴스 생성
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL', 'rougeLsum'])

  # 모델의 예측 정보 파싱
  predictions, labels = eval_pred

  # 모델 예측의 각 토큰의 대한 logit을 토큰으로 변환
  # prediction shape : (batch, max_length, 51200)
  # 마지막 차원(51200)은 어휘 사전 크기 만큼의 logit
  # np.argmax() 를 활용해 가장 확률 높은 token id로 변환
  predictions = np.argmax(predictions, axis=-1)

  # 모델 예측 token id sequence -> 텍스트
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  # 전체 batch 크기 만큼의 rouge score 종합
  for pred, ref in zip(decoded_preds, decoded_labels):
      score = scorer.score(ref, pred)
      for rouge_type in scores.keys():
          scores[rouge_type].append(score[rouge_type].fmeasure)

  # batch만큼의 평균 rouge score 계산
  avg_scores = {rouge_type: sum(score_list) / len(score_list) * 100 for rouge_type, score_list in scores.items()}

  return avg_scores

In [ ]:
trainer = SFTTrainer(
    model=flm,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    peft_config=peft_config,
    tokenizer=tokenizer,
    args = training_arguments,
    compute_metrics=compute_metrics
)


for name, module in trainer.model.named_modules():
    if 'norm' in name:
        module = module.to(torch.float32)

In [ ]:
flm.config.use_cache = False
trainer.train()

## 6. Compare between PLM and FLM

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# 예제 입력 텍스트
input_text = """
##Instruction : 삼성전자는 메모리 반도체 분야에서 세계적인 리더로 자리잡고 있습니다. 최근 HBM(고대역폭 메모리) 기술 경쟁력을 회복하며 시장에서의 입지를 강화하고 있습니다. 이는 AI 및 데이터 센터 수요 증가에 따른 것으로, 삼성전자는 이러한 수요에 대응하기 위해 기술 개발에 박차를 가하고 있습니다. 이 텍스트를 요약해줘

##Response : 
"""

# FLM 모델 사용
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)

# 입력 텐서를 GPU로 이동
inputs = {key: value.to(device) for key, value in inputs.items()}

# 모델 실행
output = flm.generate(**inputs, max_length=200)
result = tokenizer.decode(output[0], skip_special_tokens=True)
print("Output:", result)

In [ ]:
# 예제 입력 텍스트
input_text = "삼성전자는 메모리 반도체 분야에서 세계적인 리더로 자리잡고 있습니다. 최근 HBM(고대역폭 메모리) 기술 경쟁력을 회복하며 시장에서의 입지를 강화하고 있습니다. 이는 AI 및 데이터 센터 수요 증가에 따른 것으로, 삼성전자는 이러한 수요에 대응하기 위해 기술 개발에 박차를 가하고 있습니다. 이 텍스트를 확장해줘"

# plm 모델 사용
inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
# 입력 텐서를 GPU로 이동
inputs = {key: value.to(device) for key, value in inputs.items()}

output = plm.generate(**inputs, max_length=1024)
result = tokenizer.decode(output[0], skip_special_tokens=True)
print("Output:", result)